In [1]:
import brightway2 as bw
import numpy as np
from copy import copy, deepcopy

# Setup and TODOs

### TODO -> choose project

In [2]:
project = 'Agribalyse import'
bw.projects.set_current(project)

In [3]:
bw.databases

Databases dictionary with 0 objects

### TODO -> give paths to ecoinvent and agribalyse files

In [4]:
ei36_path = 'link/to/ecoinvent 3.6 cutoff/datasets'
ag13_path = 'Agribalyse CSV FINAL_no links_Nov2016v3.CSV' # need to apply for this file

# Import databases

## 1. Biosphere3

In [5]:
%%time
bw.bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.00 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/19/2020 14:47:43
  Finished: 03/19/2020 14:47:43
  Total time elapsed: 00:00:00
  CPU %: 100.30
  Memory %: 1.00
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 1.04 seconds
Wrote 850 LCIA methods with 219059 characterization factors
Creating core data migrations

CPU times: user 1min 5s, sys: 2.83 s, total: 1min 8s
Wall time: 1min 8s


## 2. Ecoinvent 3.6 cutoff

In [6]:
%%time
ei36_name = "ecoinvent 3.6 cutoff"

if ei36_name in bw.databases:
    print(ei36_name + " datasbase already present!!! No import is needed")    
else:
    ei36 = bw.SingleOutputEcospold2Importer(ei36_path, ei36_name)
    ei36.apply_strategies()
    ei36.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei36.statistics()
    if len(list(ei36.unlinked))==0:
        ei36.write_database()
    else:
        print('Unlinked exchanges!')

Extracting XML data from 18121 datasets
Extracted 18121 datasets in 19.71 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


Title: Writing activities to SQLite3 database:
  Started: 03/19/2020 14:49:18
  Finished: 03/19/2020 14:50:02
  Total time elapsed: 00:00:44
  CPU %: 95.70
  Memory %: 10.52
Created database: ecoinvent 3.6 cutoff
CPU times: user 1min 57s, sys: 10 s, total: 2min 7s
Wall time: 2min 21s


In [7]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.6 cutoff

## 3. Agribalyse 1.3


In [8]:
# Give the name of the ecoinvent database to which Agribalyse should be linked
# works smoothly for ecoinvent 3.6 cutoff, code might need modifications for other versions
ei_name = ei36_name 

In [9]:
ag13_name = 'Agribalyse 1.3'
ag13_ei_name = ag13_name + ' - ' + ei_name
ag13_ei_name

'Agribalyse 1.3 - ecoinvent 3.6 cutoff'

In [10]:
if ag13_ei_name in bw.databases:
    del bw.databases[ag13_ei_name]

### i) Following steps are similar to [Aribalyse 1.2 import](https://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20Importing%20Agribalyse%201.3%20with%20Ecoinvent%203.2%20cutoff.ipynb)

In [11]:
ag13_ei = bw.SimaProCSVImporter(ag13_path, ag13_ei_name)
ag13_ei.apply_strategies()

# Apply all migrations with previous versions of ecoinvent
ag13_ei.migrate('simapro-ecoinvent-3.3')

# Update US locations
from bw2io.strategies.locations import update_ecoinvent_locations
ag13_ei = update_ecoinvent_locations(ag13_ei)

# Biosphere flows
ag13_ei_new_biosphere_name = ag13_ei_name + " - new biosphere"
if ag13_ei_new_biosphere_name in bw.databases:
    del bw.databases[ag13_ei_new_biosphere_name]
bw.Database(ag13_ei_new_biosphere_name).register()
ag13_ei.add_unlinked_flows_to_biosphere_database(ag13_ei_new_biosphere_name)

# Add unlinked waste flows as new activities
ag13_ei.add_unlinked_activities()

ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))
ag13_ei.statistics()

Extracted 1189 unallocated datasets in 4.01 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
A

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/19/2020 14:51:41
  Finished: 03/19/2020 14:51:41
  Total time elapsed: 00:00:00
  CPU %: 107.00
  Memory %: 11.60
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1700 datasets
115259 exchanges
528 unlinked exchanges
  Type technosphere: 35 unique unlinked exchanges


(1700, 115259, 528)

### ii) Define some of the migrations manually.
### TODO -> check if migrations are correct

- These migrations are specific to agribalyse 1.3 + ecoinvent 3.6 combination
- Most of them are minor changes in names of activities and reference products
- Some activities contain `multiplier` field if the unit conversion is needed or reference products are not identical

In [12]:
agribalyse13_ecoinvent36_names = {
    'fields': ['name'],
    'data': [
        (
            ['extrusion production, plastic film'],
            {
                'name': 'extrusion, plastic film'
            }
        ),
        (
            ['extrusion production, plastic pipes'],
            {
                'name': 'extrusion, plastic pipes'
            }
        ),
        (
            ['blow moulding production'],
            {
                'name': 'blow moulding'
            }
        ),
        (
            ['calendering production, rigid sheets'],
            {
                'name': 'calendering, rigid sheets'
            }
        ),
        (
            ['steam production in chemical industry'],
            {
                'name': 'steam production, in chemical industry',
                'reference product': 'steam, in chemical industry',
                'unit': 'kilogram',
                'multiplier': 1/2.75, # change of units, see comment on this activity in ecoinvent
            }
        ),
        (
            ['water production, completely softened, from decarbonised water, at user'],
            {
                'name': 'water production, completely softened',
                'reference product': 'water, completely softened'
            }
        ),
        (
            ['market for water, decarbonised, at user'],
            {
                'name': 'market for water, decarbonised',
                'reference product': 'water, decarbonised'
            }
        ),
        (
            ['textile production, jute'],
            {
                'name': 'textile production, jute, weaving'
            }
        ),
        (
            ['market for transport, freight, sea, transoceanic ship'],
            {
                'name': 'market for transport, freight, sea, container ship with reefer, cooling',
                'reference product': 'transport, freight, sea, container ship with reefer, cooling'
            }
        ),
        (
            ['transport, freight, sea, transoceanic ship'],
            {
                'name': 'transport, freight, sea, container ship with reefer, cooling',
                'reference product': 'transport, freight, sea, container ship with reefer, cooling'
            }
        ),
        (
            ['oxidation of manganese dioxide'],
            {
                'reference product': 'potassium permanganate',
                'multiplier': 90/1000 # change of reference product, see comment on this activity in ecoinvent
            }
        ),
        (
            ['irrigation'],
            {
                'name': 'market for irrigation',
            }
        ),
    ]
}

bw.Migration("agribalyse13-ecoinvent36-names").write(
    agribalyse13_ecoinvent36_names,
    description="Change names of some activities"
)

### iii) Define migrations for market groups

In [13]:
agribalyse13_ecoinvent36_market_groups = {
    'fields': ['name', 'location'],
    'data': [
        (
            ['market for electricity, low voltage', 'BR'],
            {
                'name': 'market group for electricity, low voltage'
            }
        ),
        (
            ['market for electricity, medium voltage', 'BR'],
            {
                'name': 'market group for electricity, medium voltage'
            }
        ),
    ]
}

bw.Migration("agribalyse13-ecoinvent36-market-groups").write(
    agribalyse13_ecoinvent36_market_groups,
    description="Change markets to market groups for Brazil"
)

In [14]:
ag13_ei.migrate('agribalyse13-ecoinvent36-names')
ag13_ei.migrate('agribalyse13-ecoinvent36-market-groups')
ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))
ag13_ei.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
1700 datasets
115259 exchanges
334 unlinked exchanges
  Type technosphere: 21 unique unlinked exchanges


(1700, 115259, 334)

### iv) Allocate by production volume

In [15]:
from utils import *

In [16]:
# Create mapping between unlinked exchanges and ecoinvent activities that constitute each unlinked exchange.
# In this case we do not need to do it manually since all unlinked exchanges need to be allocated geographically.
# Example: (market for lime, GLO) is split by production volume into (market for lime, RoW) & (market for lime, RER).
# Mapping is a list of dictionaries where each dictionary corresponds to an unlinked exchange.
# The key is the name of the unlinked exchange and the values are ecoinvent activities codes.

def create_location_mapping(ag13_ei, ei_name):
    
    ei = bw.Database(ei_name)
    
    unlinked_list = list(ag13_ei.unlinked)
    len_unlinked  = len(unlinked_list)

    mapping = [0]*len_unlinked

    for u in range(len_unlinked):
        new_el = {}
        name = unlinked_list[u]['name'] 
        loc  = unlinked_list[u]['location']
        acts_codes = [act['code'] for act in ei if name == act['name']]
        new_el[ (name, loc) ] = acts_codes
        mapping[u] = new_el
        
    return mapping

In [17]:
%%time
mapping = create_location_mapping(ag13_ei, ei_name)

CPU times: user 24.2 s, sys: 7.13 s, total: 31.3 s
Wall time: 31.4 s


In [18]:
%%time
agg = modify_exchanges(ag13_ei, mapping, ei_name)

Applying strategy: link_iterable_by_fields
CPU times: user 5.29 s, sys: 364 ms, total: 5.65 s
Wall time: 5.66 s


In [19]:
agg.statistics()

1700 datasets
115943 exchanges
0 unlinked exchanges
  


(1700, 115943, 0)

### v) Change incorrect lognormal uncertainty information
So far, all exchanges with lognormal distributions have `loc` parameter equal to `amount`, instead of `log(amount)` as required by `stats_arrays`. <br>
TODO -> Ideally need to check that all distributions make sense

In [20]:
import stats_arrays as sa

In [21]:
changed = []
for i,act in enumerate(agg.data):
    excs = act.get('exchanges', [])
    for j,exc in enumerate(excs):
        if exc.get('uncertainty type', False) == sa.LognormalUncertainty.id and \
        np.allclose(exc.get('amount'), exc.get('loc')):
            exc.update(loc=np.log(exc['amount']))
            changed.append([i,j])

In [22]:
len(changed)

319

### vi) Write database
It seems that now we can write the database since there are 0 unlinked exchanges, but if we actually try then we'll get an error

In [23]:
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    agg.write_database()

NonuniqueCode: The following activities have non-unique codes: ['ammonium nitrate phosphate production', 'diammonium phosphate production', 'monoammonium phosphate production']

### vii) Fix non-unique codes

In [24]:
a = 'ammonium nitrate phosphate production'
b = 'diammonium phosphate production'
c = 'monoammonium phosphate production'

In [25]:
for act in agg.data:
    if act['name'] in [a,b,c]:
        print(act)

{'categories': ('Materials/fuels',), 'name': 'ammonium nitrate phosphate production', 'unit': 'kilogram', 'comment': '(1,1,5,1,1,na); assimilation MAP', 'location': 'RER', 'type': 'process', 'code': 'de221991cc69f37976042f05f448c94c', 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'}
{'categories': ('Materials/fuels',), 'name': 'ammonium nitrate phosphate production', 'unit': 'kilogram', 'comment': '(1,1,5,1,1,na); assimilation MAP', 'location': 'RER', 'type': 'process', 'code': 'de221991cc69f37976042f05f448c94c', 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'}
{'categories': ('Materials/fuels',), 'name': 'diammonium phosphate production', 'unit': 'kilogram', 'comment': "Mineral fertilizers. Model of transport: 'MT MAP DAP', weight transported in tons = 5,9E-02. Pedigree-Matrix='(3,3,2,1,2,na)'.", 'location': 'RER', 'type': 'process', 'code': '64037e162f6d6d1048470c3a1135f4fb', 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'}
{'categories': ('Materials/fuels',), 'name': 'di

In [26]:
# Remove repetitive activities
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'ammonium nitrate phosphate production', 
                 'unit': 'kilogram', 
                 'comment': '(1,1,5,1,1,na); assimilation MAP', 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': 'de221991cc69f37976042f05f448c94c', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'diammonium phosphate production', 
                 'unit': 'kilogram', 
                 'comment': "Mineral fertilizers. Model of transport: 'MT MAP DAP', weight transported in tons = 5,9E-02. Pedigree-Matrix='(3,3,2,1,2,na)'.", 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': '64037e162f6d6d1048470c3a1135f4fb', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'monoammonium phosphate production', 
                 'unit': 'kilogram', 
                 'comment': '', 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': '6d61eb45c1d285073770aa839426d97c', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})


### viii) Finally, write database

In [27]:
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    agg.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Title: Writing activities to SQLite3 database:
  Started: 03/19/2020 14:52:50
  Finished: 03/19/2020 14:52:59
  Total time elapsed: 00:00:08
  CPU %: 97.00
  Memory %: 11.76
Created database: Agribalyse 1.3 - ecoinvent 3.6 cutoff


In [28]:
bw.databases

Databases dictionary with 4 object(s):
	Agribalyse 1.3 - ecoinvent 3.6 cutoff
	Agribalyse 1.3 - ecoinvent 3.6 cutoff - new biosphere
	biosphere3
	ecoinvent 3.6 cutoff